# Введение в глубокое обучение с PyTorch

В этом блокноте вы познакомитесь с [PyTorch](http://pytorch.org/), фреймворком для создания и обучения нейронных сетей. Массивы в PyTorch во многом ведут себя как массивы Numpy. Обобщением массивов являются тензоры. PyTorch работает с этими тензорами и упрощает их перенос на GPU, чтобы быстрее обучать нейронные сети. Он также предоставляет модуль, который автоматически вычисляет градиенты (для обратного распространения) и еще один модуль, предназначенный специально для построения нейронных сетей. В целом, PyTorch оказывается более совместимым с Python и стеком Numpy/Scipy по сравнению с TensorFlow и другими фреймворками.



## Нейронные сети

Глубокое обучение основано на искусственных нейронных сетях, которые существуют в той или иной форме с конца 1950-х годов. Сети состоят из отдельных частей, подобных нейронам, которые обычно называются узлами или просто "нейронами". Каждый узел имеет некоторое количество взвешенных входов. Эти взвешенные входы суммируются (линейная комбинация), а затем передаются на активационную функцию, чтобы получить выход узла.

<img src="assets/simple_neuron.png" width=400px>

Математически это выглядит следующим образом: 

$$
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i +b \right)
\end{align}
$$

С векторами это скалярное произведение двух векторов:

$$
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$

## Тензоры

Вычисления в нейронных сетях являются просто множеством операций линейной алгебры над *тензорами*, обобщением матриц. Вектор — это 1-мерный тензор, матрица — это 2-мерный тензор, массив с тремя индексами — это 3-мерный тензор (например, цветные изображения RGB). Основная структура данных для нейронных сетей — это тензоры, и PyTorch (как и практически каждый другой фреймворк глубокого обучения) построен вокруг тензоров.

<img src="assets/tensor_examples.svg" width=600px>

Попробуем применить PyTorch для построения простой нейронной сети.

In [ ]:
# Сначала импортируем PyTorch
import torch

In [ ]:
def activation(x):
    """ Сигмоидная Функция активации
    
        Аргументы
        ---------
        x: torch.Tensor
    """
    return 1/(1+torch.exp(-x))

In [ ]:
### Генерация данных
torch.manual_seed(7) # Устанавливаем seed для воспроизводимости результатов

# Признаки — это 5 случайных переменных из нормального распределения
features = torch.randn((1, 5))
# Истинные (true) веса для наших данных, снова случайные переменные из нормального распределения
weights = torch.randn_like(features)
# и истинное (true) смещение
bias = torch.randn((1, 1))

Выше сгенерированы данные, которые мы можем использовать, чтобы получить выход нашей простой сети. На данный момент все это просто случайные данные из нормального распределения. Пройдемся по каждой строке:

`features = torch.randn((1, 5))` создает тензор с формой `(1, 5)`, одной строкой и пятью столбцами, который содержит значения, случайно распределенные в соответствии с нормальным распределением со средним значением ноль и стандартным отклонением один. 

`weights = torch.randn_like(features)` создает другой тензор с такой же формой, как и `features`, снова содержащий значения из нормального распределения.

Наконец, `bias = torch.randn((1, 1))` создает одно значение из нормального распределения.

Тензоры PyTorch можно складывать, умножать, вычитать и так далее, точно так же, как и массивы Numpy. В общем, вы будете использовать тензоры PyTorch практически так же, как вы бы использовали массивы Numpy. Однако они имеют несколько приятных преимуществ, таких как ускорение с помощью GPU, к которому мы перейдем позже. А пока используйте сгенерированные данные для вычисления выхода этой простой однослойной сети. 
> **Упражнение**: Вычислите выход сети с входными признаками `features`, весами `weights` и смещением `bias`. Подобно Numpy, PyTorch имеет функцию [`torch.sum()`](https://pytorch.org/docs/stable/torch.html#torch.sum), а также метод `.sum()` на тензорах для суммирования. Используйте функцию `activation`, определенную выше, в качестве функции активации.

In [ ]:
## TODO


Вы можете произвести умножение и суммирование в одной операции, используя матричное умножение. В общем случае, вы предпочтительно использовать матричное умножение, так как оно более эффективно и ускоряется с помощью современных библиотек и высокопроизводительных вычислений на GPU.

Попробуем сделать матричное умножение признаков и весов. Для этого мы можем использовать [`torch.mm()`](https://pytorch.org/docs/stable/torch.html#torch.mm) или [`torch.matmul()`](https://pytorch.org/docs/stable/torch.html#torch.matmul), которые несколько более сложные. Если мы попробуем сделать это с `features` и `weights` в их текущем виде, мы получим ошибку

```python
>> torch.mm(features, weights)

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-13-15d592eb5279> in <module>()
----> 1 torch.mm(features, weights)

RuntimeError: size mismatch, m1: [1 x 5], m2: [1 x 5] at /Users/soumith/minicondabuild3/conda-bld/pytorch_1524590658547/work/aten/src/TH/generic/THTensorMath.c:2033
```

При создании нейронных сетей в любом фреймворке вы будете довольно часто сталкиваться с этим типом ошибок. В данном случае наши тензоры не имеют правильных размерностей для выполнения матричного умножения. Вспомним, что для матричного умножения количество столбцов в первом тензоре должно быть равно количеству строк во втором тензоре. Оба `features` и `weights` имеют одну и ту же форму (shape) `(1, 5)`. Это означает, что нам нужно изменить форму `weights`, чтобы выполнить матричное умножение.

**Примечание:** Чтобы увидеть форму тензора, называемого `tensor`, используйте `tensor.shape`. При построении нейронных сетей вы будете часто использовать этот метод.

Есть несколько вариантов, как изменить размеры (shape) тензора: [`weights.reshape()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.reshape), [`weights.resize_()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.resize_), и [`weights.view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view).

* `weights.reshape(a, b)` вернет новый тензор с теми же данными (иногда), что и `weights`, размером `(a, b)`, а иногда клона, так как копирует данные в другую часть памяти.
* `weights.resize_(a, b)` возвращает тот же тензор другого размера. Однако, если новый размер приводит к меньшему количеству элементов, чем оригинальный тензор, некоторые элементы будут удалены из тензора (но не из памяти). Если новая форма приводит к большему количеству элементов, чем оригинальный тензор, новые элементы будут неинициализированными в памяти. Подчеркивание в конце метода указывает на то, что этот метод выполняется **на месте**. Вот отличная тематическая беседа, чтобы [узнать больше о операциях на месте](https://discuss.pytorch.org/t/what-is-in-place-operation/16244) в PyTorch.
* `weights.view(a, b)` вернет новый тензор с теми же данными, что и `weights`, размером `(a, b)`.

Обычно используется `.view()`, но любой из трех методов подойдет для этой задачи. Итак, теперь мы можем изменить форму `weights`, чтобы иметь пять строк и один столбец, применив что-то вроде `weights.view(5, 1)`.

> **Упражнение**: Вычислите выход нашей маленькой сети, используя матричное умножение.

In [ ]:
## TODO


### Сложите их вместе

Вы можете вычислить выход для одного нейрона. Реальная мощь этого алгоритма проявляется, когда вы начинаете объединять эти отдельные узлы в слои и наборы слоев, в сеть нейронов. Выход одного слоя нейронов становится входом для следующего слоя. Теперь нам нужно выразить веса в виде матрицы.

<img src='assets/multilayer_diagram_weights.png' width=450px>

Первый слой, показанный внизу, это входы, называемые **входным слоем**. Средний слой называется **скрытым слоем**, а последний слой (справа) — **выходным слоем**. Мы можем выразить эту сеть математически с помощью матриц и использовать матричное умножение, чтобы получить линейные комбинации для каждой узла в одной операции. Например, скрытый слой ($h_1$ и $h_2$ здесь) можно вычислить 

$$
\vec{h} = [h_1 \, h_2] = 
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

Выход для этой маленькой сети найдем, рассматривая скрытый слой как входы для выходного узла. Выход сети выражается как

$$
y =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$

In [ ]:
### Генерация данных
torch.manual_seed(7) # Устанавливаем seed для воспроизводимости результатов

# Признаки — это 3 случайные переменные из нормального распределения
features = torch.randn((1, 3))

# Определим размер каждого слоя в нашей сети
n_input = features.shape[1]     # Количество входных узлов, должно совпадать с количеством входных признаков
n_hidden = 2                    # Количество скрытых узлов 
n_output = 1                    # Количество выходных узлов

# Веса для входов к скрытому слою
W1 = torch.randn(n_input, n_hidden)
# Веса для скрытого слоя к выходному слою
W2 = torch.randn(n_hidden, n_output)

# и смещения для скрытого и выходного слоев
B1 = torch.randn((1, n_hidden))
B2 = torch.randn((1, n_output))

> **Упражнение:** Вычислите выход для этой многослойной сети, используя веса `W1` и `W2`, а также смещения `B1` и `B2`. 

In [ ]:
## TODO


Если вы сделали это правильно, то должны увидеть выход `tensor([[ 0.3171]])`.

Количество скрытых узлов — это параметр сети, часто называемый **гиперпараметром**, чтобы отличать его от параметров весов и смещения. Как вы увидите позже, когда мы будем обсуждать обучение нейронной сети, чем больше скрытых единиц имеет сеть и чем больше слоев, тем лучше она может учиться на данных и делать более точные предсказания.

## Numpy в Torch и обратно

PyTorch имеет удобные функции для преобразования между массивами Numpy и тензорами Torch. Чтобы создать тензор из массива Numpy, используйте `torch.from_numpy()`. Чтобы преобразовать тензор в массив Numpy, используйте метод `.numpy()`.

In [ ]:
import numpy as np
a = np.random.rand(4,3)
a

In [ ]:
b = torch.from_numpy(a)
b

In [ ]:
b.numpy()

Память разделяется между массивом Numpy и тензором Torch, так что если вы измените значения на месте (in-place) одного объекта, другой также изменится.

In [ ]:
# Умножаем тензор PyTorch на 2, in-place
b.mul_(2)

In [ ]:
# Массив Numpy соответствует новому значению из Тензора
a